This notebook `.ipynb` aims to do analysis on the `results/mlama_factual_knowledge_results` result from the `experiments/factual_knowledge_experiment` experiment.

In [3]:
import pandas as pd

In [4]:
result_df = pd.read_csv("../results/mlama_factual_knowledge_results.csv")
result_df

,language,subject,relation,question,options,golden_answer,predicted_answer,is_correct
0,afr,President van Frankryk,P1001,President van Frankryk is 'n wettige term in _...,"['Nigerië', 'Italië', 'Jamaika', 'Frankryk']",Frankryk,Frankryk,True
1,afr,Kategorie:Eerste Ministers van Kanada,P1001,Kategorie:Eerste Ministers van Kanada is 'n we...,"['Kanada', 'Guyana', 'Indonesië', 'Bolivië']",Kanada,Kanada,True
2,afr,President van Brasilië,P1001,President van Brasilië is 'n wettige term in _...,"['Libië', 'Japan', 'Skotland', 'Brasilië']",Brasilië,Brasilië,True
3,afr,Eerste Minister van Australië,P1001,Eerste Minister van Australië is 'n wettige te...,"['Kuba', 'Viëtnam', 'Australië', 'Turkye']",Australië,Australië,True
4,afr,Parlement van Kanada,P1001,Parlement van Kanada is 'n wettige term in _____.,"['Bolivië', 'Kanada', 'Nigerië', 'Haïti']",Kanada,Kanada,True
...,...,...,...,...,...,...,...,...
10795,zho,羅馬荷蘭法,P1001,羅馬荷蘭法是_____中的合法术语。,"['塞尔维亚', '白俄罗斯', '波札那', '日本']",波札那,波札那,True
10796,zho,羅馬荷蘭法,P1001,羅馬荷蘭法是_____中的合法术语。,"['中華民國', '得克萨斯州', '辛巴威', '塔斯馬尼亞州']",辛巴威,中華民國,False
10797,zho,羅馬荷蘭法,P1001,羅馬荷蘭法是_____中的合法术语。,"['阿尔及利亚', '马来西亚', '圭亚那', '丹麦']",圭亚那,丹麦,False
10798,zho,羅馬荷蘭法,P1001,羅馬荷蘭法是_____中的合法术语。,"['底特律', '印度尼西亚', '泰国', '特伦甘纳邦']",印度尼西亚,印度尼西亚,True


In [5]:
result_df.head()

,language,subject,relation,question,options,golden_answer,predicted_answer,is_correct
0,afr,President van Frankryk,P1001,President van Frankryk is 'n wettige term in _...,"['Nigerië', 'Italië', 'Jamaika', 'Frankryk']",Frankryk,Frankryk,True
1,afr,Kategorie:Eerste Ministers van Kanada,P1001,Kategorie:Eerste Ministers van Kanada is 'n we...,"['Kanada', 'Guyana', 'Indonesië', 'Bolivië']",Kanada,Kanada,True
2,afr,President van Brasilië,P1001,President van Brasilië is 'n wettige term in _...,"['Libië', 'Japan', 'Skotland', 'Brasilië']",Brasilië,Brasilië,True
3,afr,Eerste Minister van Australië,P1001,Eerste Minister van Australië is 'n wettige te...,"['Kuba', 'Viëtnam', 'Australië', 'Turkye']",Australië,Australië,True
4,afr,Parlement van Kanada,P1001,Parlement van Kanada is 'n wettige term in _____.,"['Bolivië', 'Kanada', 'Nigerië', 'Haïti']",Kanada,Kanada,True


In [6]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10800 entries, 0 to 10799
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   language          10400 non-null  object
 1   subject           10800 non-null  object
 2   relation          10800 non-null  object
 3   question          10800 non-null  object
 4   options           10800 non-null  object
 5   golden_answer     10800 non-null  object
 6   predicted_answer  10800 non-null  object
 7   is_correct        10800 non-null  bool  
dtypes: bool(1), object(7)
memory usage: 601.3+ KB
